# Generative Adversarial Network(GAN)

***ABSTRACT***

This is a hands-on practice of using Keras and Tensorflow to build and train a Generative Adversarial Network(GAN) against MNIST dataset which has handwritten digit black/white images with 28x28 pixels. This dataset is widely known for for machine learning purpose. We first load the dataset from Keras library and pre-process it. Then we build a Generator and a Discriminator, we also build a GAN which connect the Generator with the Discriminator. We take 10 digit image from the MNIST handwritten dataset as ground truth sample to train our GAN. We create a random noise vector with Gaussian distribution and feed it to the Generator to create a number of generated images and label them as 0, we also use the ground truth images and label them as 1, we use these images to feed to  Discriminator and train it. After the Discriminator is trained, we again create random noise and label them as 1 to fool the Discriminator, and train the GAN with Discriminator frozen, the purpose of this is to train the Generator and fool the Discriminator to make it think the generated image are real. We run these steps repeatly until a number of epoches, during running we visulize the generated images which give an animated effect to show how the generated images are improved as the training process is going.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

Using TensorFlow backend.


# 1. MNIST Dataset

The MNIST database is a well known dataset that contains thousands of handwritten digits for machine learning purpose.

## 1.1 Load MNIST dataset from tensorflow library
When we load the dataset below, X_train and X_test will contain the images, and y_train and y_test will contain the digits that those images represent.

In [0]:
#(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
#print("X_train", X_train.shape)
#print("y_train", y_train.shape)
#print("X_test", X_test.shape)
#print("y_test", y_test.shape)


In [2]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5

    
    # convert shape of x_train from (60000, 28, 28) to (60000, 784) 
    # 784 columns per row
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)
  
(X_train, y_train,X_test, y_test)=load_data()
print(X_train.shape)

(60000, 784)


## 1.2 Pre-process the data

In [0]:
#X_train = X_train.astype(np.float32).reshape(X_train.shape[0], height, width, channels)
#X_test = X_test.astype(np.float32).reshape(X_test.shape[0], height, width, channels)
#input_shape = (height, width, 1)

# 2. Build Generative Adversarial Network(GAN)

## 2.1 Build a Generator

In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [5]:
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=256,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=784, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator
  
theGenerator = create_generator()
theGenerator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dens

## 2.2 Build a Discriminator

In [6]:
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
  
theDiscriminator = create_discriminator()
theDiscriminator.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
___________________________

## 2.3 Build the GAN by connecting the Generator and Discriminator

In [7]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
  
theGan = create_gan(theDiscriminator, theGenerator)
theGan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


# 3. Training the GAN

## 3.1 Visualize the original images and generated images

In [0]:
import numpy as np
from IPython.display import SVG, display
from PIL import Image

def invert_image(img):
    return( np.invert(img)  )

  
def denomalize_image(img, invert=False):
    if img.dtype is np.dtype(np.float32):
      image = img * 255
      image = image.astype('uint8')
    else:
      image = img
    
    if invert == True:
      image = invert_image( image )

    return( image )
    

def init_display(img, zoom=1):
    height, width = img.shape
    img_array=denomalize_image(img, invert=True)
    img_disp = Image.fromarray(img_array,mode='P')
    if zoom == 1:
      out = display(img_disp, display_id=True)
    else:
      out = display(img_disp.resize((width*zoom,height*zoom)), display_id=True)
    return( out )


def update_display(out, img, zoom=1):
    height, width = img.shape
    img_array=denomalize_image(img, invert=True)
    img_disp = Image.fromarray(img_array,mode='P')
    if zoom == 1:
      out.update(img_disp)
    else:
      out.update(img_disp.resize((width*zoom,height*zoom)))


      
def plot_output( epoch, generator, examples=16, initial=False, out=None ):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples,28,28)
    plot_image = np.concatenate(generated_images, axis = 1)
    if initial == False:
      update_display(out, plot_image)
    else:
      output = init_display(plot_image)
      return( output )
    

def plot_sample( sample ):
    plot_image = np.concatenate(sample.reshape(sample.shape[0],28,28), axis = 1)
    output = init_display(plot_image)
    return( output )
        
    
    

## 3.2 Train the GAN

In [17]:
epochs = 200
batch_size = 64
sample_size = 10

#Pick the sample images -- each one from number 0 to 9
sample = np.empty(shape=sample_size).astype('uint8')
for i in range(0,sample_size):
  sample[i] = np.where(y_train==i)[0][1]

#Show the sample images
sample_image = X_train[sample]
plot_sample( sample_image )


#Display the initial generator generated images
display_output = plot_output( 0, theGenerator, examples=16, initial=True )


#
# Main loop for training the GAN
#
for e in range(1,epochs+1 ):
    print("Epoch %d" %e)
    for _ in tqdm(range(batch_size)):
      
        #generate random noise as an input to initialize the generator
        noise= np.random.normal(0,1, [batch_size, 100])
        generated_images = theGenerator.predict(noise)

        # Make the ground truth images from the sample images, 
        # randomly create them by the number of batch_size 
        #image_batch = X_train[np.random.choice(sample, size=batch_size)]
        image_batch = sample_image

        #Mix the ground truth images with the generated images,
        #and label them 1 for ground truth and 0 for generated ones
        X_gan = np.concatenate([image_batch, generated_images])
        y_gan = np.concatenate([np.ones(image_batch.shape[0]), np.zeros(generated_images.shape[0])])
        y_gan = y_gan.astype(np.float32)

        #First train the discriminator. 
        theDiscriminator.trainable=True
        theDiscriminator.train_on_batch(X_gan, y_gan)

        #Tricking the noised input of the Generator as real data
        noise= np.random.normal(0,1, [batch_size, 100])
        y_gen = np.ones(batch_size)

        # During the training of gan, 
        # the weights of discriminator should be fixed. 
        #We can enforce that by setting the trainable flag
        theDiscriminator.trainable=False

        #training  the GAN by alternating the training of the Discriminator 
        #and training the chained GAN model with Discriminator’s weights freezed.
        theGan.train_on_batch(noise, y_gen)

    if e == 1 or e % 2 == 0:
        #plot_generated_images(e, generator)
        plot_output( e, theGenerator, out=display_output )



 14%|█▍        | 9/64 [00:00<00:00, 81.48it/s]

Epoch 1


 14%|█▍        | 9/64 [00:00<00:00, 89.10it/s]

Epoch 2


 16%|█▌        | 10/64 [00:00<00:00, 92.92it/s]

...

 17%|█▋        | 11/64 [00:00<00:00, 103.33it/s]

Epoch 199


 17%|█▋        | 11/64 [00:00<00:00, 102.76it/s]

Epoch 200


100%|██████████| 64/64 [00:00<00:00, 101.45it/s]


## 3.3 Display the results

In [23]:
plot_sample( sample_image )

for i in range(16):
  plot_output( 0, theGenerator, examples=16, initial=True )

# 4. Others, for testing purpose


Reference:
* https://medium.com/datadriveninvestor/generative-adversarial-network-gan-using-keras-ce1c05cfdfd3
* https://blog.insightdatascience.com/generating-custom-photo-realistic-faces-using-ai-d170b1b59255




In [11]:
#Pick the sample images
sample = np.empty(shape=10).astype('uint8')
for i in range(0,10):
  sample[i] = np.where(y_train==i)[0][5]

image_batch = X_train[sample]
plot_sample( image_batch )


<DisplayHandle display_id=9992b83fe79c61a9b864fc1f5cae043c>

In [12]:
plot_sample( X_gan[0:20,:] )

<DisplayHandle display_id=31777bb72b1852e015af468e35b8161a>

In [13]:
y_gan

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)